## Import the dependencies

In [1]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.4 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from spacy.tokenizer import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import spacy
import re
import numpy as np
import string
import matplotlib.pyplot as plt
import csv
import contractions
from nltk.corpus import stopwords
from spacy.tokenizer import Tokenizer
import random
from tqdm import tqdm
import math
import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using",device)

Using cuda


## Load the data

In [7]:
df1 = pd.read_csv("/content/news_summary.csv",encoding='latin-1',usecols=['headlines','text'])
df2 = pd.read_csv("/content/news_summary_more.csv",encoding='iso-8859-1')

In [ ]:
df1.head(5)

,headlines,text
0,Daman & Diu revokes mandatory Rakshabandhan in offices order,The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.
1,Malaika slams user who trolled her for 'divorcing rich man',"Malaika Arora slammed an Instagram user who trolled her for ""divorcing a rich man"" and ""having fun with the alimony"". ""Her life now is all about wearing short clothes, going to gym or salon, enjoying vacation[s],"" the user commented. Malaika responded, ""You certainly got to get your damn facts right before spewing sh*t on me...when you know nothing about me."""
2,'Virgin' now corrected to 'Unmarried' in IGIMS' form,"The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna on Thursday made corrections in its Marital Declaration Form by changing 'Virgin' option to 'Unmarried'. Earlier, Bihar Health Minister defined virgin as being an unmarried woman and did not consider the term objectionable. The institute, however, faced strong backlash for asking new recruits to declare their virginity in the form."
3,Aaj aapne pakad liya: LeT man Dujana before being killed,"Lashkar-e-Taiba's Kashmir commander Abu Dujana, who was killed by security forces, said ""Kabhi hum aage, kabhi aap, aaj aapne pakad liya, mubarak ho aapko (Today you caught me. Congratulations)"" after being caught. He added that he won't surrender, and whatever is in his fate will happen to him. ""Hum nikley they shaheed hone (had left home for martyrdom),"" he added."
4,Hotel staff to get training to spot signs of sex trafficking,"Hotels in Maharashtra will train their staff to spot signs of sex trafficking, including frequent requests for bed linen changes and 'Do not disturb' signs left on room doors for days. A mobile phone app called Rescue Me, which will allow staff to alert police of suspicious behaviour, will be developed. The initiative has been backed by the Maharashtra government."


In [ ]:
df1.shape

(4514, 2)

In [ ]:
df2.head()

,headlines,text
0,upGrad learner switches to career in ML & Al with 90% salary hike,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers."
1,Delhi techie wins free food from Swiggy for one year on CRED,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more."
2,New Zealand end Rohit Sharma-led India's 12-match winning streak,"New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The match witnessed India getting all out for 92, their seventh lowest total in ODI cricket history."
3,Aegon life iTerm insurance plan helps customers save tax,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢ÂÂ¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, customers have options to insure against Critical Illnesses, Disability and Accidental Death Benefit Rider with a life cover up to the age of 80 years."
4,"Have known Hirani for yrs, what if MeToo claims are not true: Sonam","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""In the #MeToo movement, I always believe a woman. But in this case, we need to reserve our judgment,"" she added. Hirani has been accused by an assistant who worked in 'Sanju'."


In [ ]:
df2.shape

(98401, 2)

In [8]:
train_df = df2.copy()
test_df = df1.copy()

In [ ]:
train_df.describe()

,headlines,text
count,98401,98401
unique,98280,98360
top,Warne rightly predicted India-Eng 2011 WC match to be a tie,"Former Pakistani cricketer Hanif Mohammad batted for 970 minutes in a Test against West Indies, during his knock of 337 runs, which ended on January 23, 1958. The innings, covering nine consecutive sessions and enough to encompass over 10 football matches, included 24 boundaries, 105 singles, 44 twos and 16 threes. The 337-run knock helped Pakistan draw the match."
freq,3,2


In [ ]:
test_df.describe()

,headlines,text
count,4514,4514
unique,4514,4514
top,More than half of India's languages may die in 50yrs: Survey,"At least 400 languages or more than half languages spoken by India's 1.3 billion people may die out in next 50 years, a study by the People's Linguistic Survey of India (PSLI) stated. Further claiming that India has already lost 250 languages in last five decades, the survey said, ""Each time a language is lost, the corresponding culture is killed."""
freq,1,1


In [9]:
## Looks like training data has few duplicate rows. Let's drop these

train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True,subset=['headlines'])
train_df.drop_duplicates(inplace=True,subset=['text'])

In [ ]:
train_df.describe()

,headlines,text
count,98262,98262
unique,98262,98262
top,Madhesi Morcha withdraws support to Nepalese government,"Madhesi Morcha, an alliance of seven political parties, has withdrawn support to PM Pushpa Kamal Dahal-led Nepal government after it failed to meet a seven-day ultimatum to fulfil their demands including endorsement for the revised Constitution amendment bill. The Morcha has 36 seats in the Parliament, but despite the withdrawal of support, there is no immediate threat to the government."
freq,1,1


## Clean/Preprocess Text

In [10]:
train_df = train_df.rename(columns={'headlines':'target_text','text':'source_text'})
test_df = test_df.rename(columns={'headlines':'target_text','text':'source_text'})

In [11]:
print(train_df.columns)
print(test_df.columns)

Index(['target_text', 'source_text'], dtype='object')
Index(['target_text', 'source_text'], dtype='object')


In [12]:
### Let's try to remove short forms like I'll -> I will etc.

train_df['source_text'] = train_df['source_text'].apply(lambda x: contractions.fix(x))
train_df['target_text'] = train_df['target_text'].apply(lambda x: contractions.fix(x))

test_df['target_text'] = test_df['target_text'].apply(lambda x: contractions.fix(x))
test_df['target_text'] = test_df['target_text'].apply(lambda x: contractions.fix(x))

In [13]:
stop_words = set(stopwords.words('english'))

In [14]:
def cleantext(text):
    new_text = str(text)
    new_text = new_text.split()
    words=[]
    for t in new_text:
        if t.isalpha():
            words.append(t)
    new_text =" ".join(words)
    new_text = new_text.lower()
    new_text = re.sub(r" e g "," eg ",new_text)
    new_text = re.sub(r"e-mail","email",new_text)
    new_text = re.sub(r"9\\/11"," 911 ",new_text)
    new_text = re.sub(r" u.s"," american ",new_text)
    new_text = re.sub(r" u.n"," united nations ",new_text)
    new_text = re.sub(r"\n"," ",new_text)
    new_text = re.sub(r":"," ",new_text)
    new_text = re.sub(r"-"," ",new_text)
    new_text = re.sub(r"\_"," ",new_text)
    new_text = re.sub(r"\d+"," ",new_text)
    new_text = re.sub(r"[$#@%&*!~?%{}()]"," ",new_text)

    tokens = [w for w in new_text.split() if not w in stop_words]


    return " ".join(tokens)

In [15]:
## Apply the transformations on source and target texts
for cols in train_df.columns:
    train_df[cols] = train_df[cols].apply(cleantext)

for cols in test_df.columns:
    test_df[cols] = test_df[cols].apply(cleantext)

In [ ]:
train_df.head()

,target_text,source_text
0,upgrad learner switches career ml al salary hike,saurav alumnus upgrad pg program machine learning artificial sr systems engineer infosys almost years work program career support helped transition data scientist tech mahindra salary online power learning powered
1,delhi techie wins free food swiggy one year cred,kunal credit card bill payment gave users chance win free food swiggy one pranav delhi bagged reward spending cred users get one cred coin per rupee bill used avail rewards brands like
2,new zealand end rohit winning streak,new zealand defeated india wickets fourth odi hamilton thursday win first match odi india lost international match rohit captaincy consecutive victories dating back march match witnessed india getting seventh lowest total odi cricket
3,aegon life iterm insurance plan helps customers save tax,aegon life iterm insurance customers enjoy tax benefits premiums paid save plan provides life cover age customers options insure critical disability accidental death benefit rider life cover age
4,known hirani metoo claims sonam,speaking sexual harassment allegations rajkumar sonam kapoor known hirani many movement get always believe need reserve hirani accused assistant worked


In [ ]:
train_df.describe()

,target_text,source_text
count,98262,98262
unique,98038,98232
top,,legend bradman smashed runs help australia post india test ended december india dismissed first innings second combined score could overhaul individual highest score india match chandu
freq,9,2


In [ ]:
test_df.head()

,target_text,source_text
0,daman diu revokes mandatory rakshabandhan offices order,administration union territory daman diu revoked order made compulsory women tie rakhis male colleagues occasion rakshabandhan august administration forced withdraw decision within hours issuing circular received flak employees slammed social
1,malaika slams user trolled rich,malaika arora slammed instagram user trolled rich fun life wearing short going gym enjoying user malaika certainly got get damn facts right spewing know nothing
2,corrected form,indira gandhi institute medical sciences patna thursday made corrections marital declaration form changing option bihar health minister defined virgin unmarried woman consider term faced strong backlash asking new recruits declare virginity
3,aaj aapne pakad let man dujana killed,kashmir commander abu killed security said hum kabhi aaj aapne pakad mubarak ho aapko caught added whatever fate happen nikley shaheed hone left home
4,hotel staff get training spot signs sex trafficking,hotels maharashtra train staff spot signs sex including frequent requests bed linen changes signs left room doors mobile phone app called rescue allow staff alert police suspicious initiative backed maharashtra


In [ ]:
test_df.describe()

,target_text,source_text
count,4514,4514
unique,4512,4514
top,pakistan arrests indian fishermen gujarat coast,least languages half languages spoken billion people may die next study linguistic survey india claiming india already lost languages last five survey time language corresponding culture
freq,2,1


In [16]:
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)

In [17]:
## Tokenize the data using NLTK

for cols in train_df.columns:
    train_df[cols] = train_df[cols].apply(lambda x: [token.text.lower() for token in tokenizer(x)])

for cols in test_df.columns:
    test_df[cols] = test_df[cols].apply(lambda x: [token.text.lower() for token in tokenizer(x)])

In [ ]:
train_df.head()

,target_text,source_text
0,"[upgrad, learner, switches, career, ml, al, salary, hike]","[saurav, alumnus, upgrad, pg, program, machine, learning, artificial, sr, systems, engineer, infosys, almost, years, work, program, career, support, helped, transition, data, scientist, tech, mahindra, salary, online, power, learning, powered]"
1,"[delhi, techie, wins, free, food, swiggy, one, year, cred]","[kunal, credit, card, bill, payment, gave, users, chance, win, free, food, swiggy, one, pranav, delhi, bagged, reward, spending, cred, users, get, one, cred, coin, per, rupee, bill, used, avail, rewards, brands, like]"
2,"[new, zealand, end, rohit, winning, streak]","[new, zealand, defeated, india, wickets, fourth, odi, hamilton, thursday, win, first, match, odi, india, lost, international, match, rohit, captaincy, consecutive, victories, dating, back, march, match, witnessed, india, getting, seventh, lowest, total, odi, cricket]"
3,"[aegon, life, iterm, insurance, plan, helps, customers, save, tax]","[aegon, life, iterm, insurance, customers, enjoy, tax, benefits, premiums, paid, save, plan, provides, life, cover, age, customers, options, insure, critical, disability, accidental, death, benefit, rider, life, cover, age]"
4,"[known, hirani, metoo, claims, sonam]","[speaking, sexual, harassment, allegations, rajkumar, sonam, kapoor, known, hirani, many, movement, get, always, believe, need, reserve, hirani, accused, assistant, worked]"


In [ ]:
test_df.head()

,target_text,source_text
0,"[daman, diu, revokes, mandatory, rakshabandhan, offices, order]","[administration, union, territory, daman, diu, revoked, order, made, compulsory, women, tie, rakhis, male, colleagues, occasion, rakshabandhan, august, administration, forced, withdraw, decision, within, hours, issuing, circular, received, flak, employees, slammed, social]"
1,"[malaika, slams, user, trolled, rich]","[malaika, arora, slammed, instagram, user, trolled, rich, fun, life, wearing, short, going, gym, enjoying, user, malaika, certainly, got, get, damn, facts, right, spewing, know, nothing]"
2,"[corrected, form]","[indira, gandhi, institute, medical, sciences, patna, thursday, made, corrections, marital, declaration, form, changing, option, bihar, health, minister, defined, virgin, unmarried, woman, consider, term, faced, strong, backlash, asking, new, recruits, declare, virginity]"
3,"[aaj, aapne, pakad, let, man, dujana, killed]","[kashmir, commander, abu, killed, security, said, hum, kabhi, aaj, aapne, pakad, mubarak, ho, aapko, caught, added, whatever, fate, happen, nikley, shaheed, hone, left, home]"
4,"[hotel, staff, get, training, spot, signs, sex, trafficking]","[hotels, maharashtra, train, staff, spot, signs, sex, including, frequent, requests, bed, linen, changes, signs, left, room, doors, mobile, phone, app, called, rescue, allow, staff, alert, police, suspicious, initiative, backed, maharashtra]"


In [18]:
## Let's try to collate the vocabulary for source and target texts

all_text = list(train_df['source_text']) + list(test_df['source_text']) + list(train_df['target_text']) + list(test_df['target_text'])

In [ ]:
all_text[0:5]

[['saurav',
  'alumnus',
  'upgrad',
  'pg',
  'program',
  'machine',
  'learning',
  'artificial',
  'sr',
  'systems',
  'engineer',
  'infosys',
  'almost',
  'years',
  'work',
  'program',
  'career',
  'support',
  'helped',
  'transition',
  'data',
  'scientist',
  'tech',
  'mahindra',
  'salary',
  'online',
  'power',
  'learning',
  'powered'],
 ['kunal',
  'credit',
  'card',
  'bill',
  'payment',
  'gave',
  'users',
  'chance',
  'win',
  'free',
  'food',
  'swiggy',
  'one',
  'pranav',
  'delhi',
  'bagged',
  'reward',
  'spending',
  'cred',
  'users',
  'get',
  'one',
  'cred',
  'coin',
  'per',
  'rupee',
  'bill',
  'used',
  'avail',
  'rewards',
  'brands',
  'like'],
 ['new',
  'zealand',
  'defeated',
  'india',
  'wickets',
  'fourth',
  'odi',
  'hamilton',
  'thursday',
  'win',
  'first',
  'match',
  'odi',
  'india',
  'lost',
  'international',
  'match',
  'rohit',
  'captaincy',
  'consecutive',
  'victories',
  'dating',
  'back',
  'march',
  '

In [19]:
## sort basis the word
source_vocab = {actual_word: idx for idx, (word_num, actual_word) in enumerate(sorted(enumerate(set(token for tokens in all_text for token in tokens)), key=lambda x: x[1]))}
target_vocab = {actual_word: idx for idx, (word_num, actual_word) in enumerate(sorted(enumerate(set(token for tokens in all_text for token in tokens)), key=lambda x: x[1]))}

In [20]:
class CustomDataset(Dataset):
    def __init__(self, source_texts, target_summaries, source_vocab, target_vocab):
        self.source_texts = source_texts
        self.target_summaries = target_summaries
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = [self.source_vocab[word] for word in self.source_texts[idx]]
        target_summary = [self.target_vocab[word] for word in self.target_summaries[idx]]
        return torch.tensor(source_text), torch.tensor(target_summary)

In [21]:
train_dataset = CustomDataset(train_df['source_text'].tolist(), train_df['target_text'].tolist(),source_vocab, target_vocab)
test_dataset = CustomDataset(test_df['source_text'].tolist(), test_df['target_text'].tolist(),source_vocab, target_vocab)

In [22]:
def get_max_seqlen():
    max_length = 0
    for index, row in train_df.iterrows():
        # Calculate the length of the current row
        row_length = len(row['source_text'])
        # Update the maximum length if the current row length is greater
        max_length = max(max_length, row_length)
    for index, row in test_df.iterrows():
        # Calculate the length of the current row
        row_length = len(row['source_text'])
        # Update the maximum length if the current row length is greater
        max_length = max(max_length, row_length)
    print("Max length in dataset ",max_length)
    return max_length

In [23]:
def collate_fn(batch):
    sources, targets = zip(*batch)
    padded_sources = pad_sequence(sources, batch_first=True)
    padded_targets = pad_sequence(targets, batch_first=True)
    return padded_sources, padded_targets

In [24]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(MultiHeadAttention,self).__init__()
        assert embedding_dim % num_heads == 0, "embedding_dim must be divisible by num_heads"

        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dim_perhead = embedding_dim // num_heads

        self.W_q = nn.Linear(embedding_dim, embedding_dim)
        self.W_k = nn.Linear(embedding_dim, embedding_dim)
        self.W_v = nn.Linear(embedding_dim, embedding_dim)
        self.W_o = nn.Linear(embedding_dim, embedding_dim)

    def scaled_dot_product_attention(self,Q,K,V,mask=None):
        # Q,K,V Shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]

        K = K.transpose(-2,-1) # K = K.permute(0,1,3,2) also works
        # K Shape(after permute) : [Batch_Size X Num_Heads X Dim Per Head X Seq_len]
        attn_scores = torch.matmul(Q,K) / math.sqrt(self.dim_perhead)
        # attn_scores Shape : [Batch_Size X Num_Heads X Seq_len X Seq_len]
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        # attn_probs Shape : [Batch_Size X Num_Heads X Seq_len X Seq_len]
        output = torch.matmul(attn_probs, V)
        # output Shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]
        return output

    def split_heads(self, x):
        # X shape : [Batch_Size X Seq_len X Embedding Dim]
        batch_size, seq_length, d_model = x.size()
        x = x.view(batch_size, seq_length,self.num_heads,self.dim_perhead)
        # X shape : [Batch_Size X Seq_len X Num_Heads X Dim Per Head]
        x = x.transpose(1,2)
        # X shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]
        return x

    def combine_heads(self, x):
        # x Shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]
        batch_size, _, seq_length, dim_perhead = x.size()
        x = x.transpose(1,2).contiguous()
        # x Shape : [Batch_Size X Seq_len X Num_Heads X Dim Per Head]
        x = x.view(batch_size, seq_length,self.embedding_dim)
        # x Shape : [Batch_Size X Seq_len X Embedding Dim]
        return x

    def forward(self, Q, K, V, mask=None):
        # Q,K,V Shape : [Batch_Size X Seq_len X Embedding Dim]
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        # Q,K,V Shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        # attn_output Shape : [Batch_Size X Num_Heads X Seq_len X Dim Per Head]
        output = self.W_o(self.combine_heads(attn_output))
        # output Shape :  # x Shape : [Batch_Size X Seq_len X Embedding Dim]
        return output

In [25]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # shape does not change here
        return self.fc2(F.relu(self.fc1(x)))


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length,dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model,2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe',pe.unsqueeze(0))

    def forward(self, x):
        # shape does not change here, adding positional encoding information
        return x + self.pe[:, :x.size(1)]

In [26]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        # x shape [Batch_Size X Seq_len X Embedding Dim]
        attn_output = self.self_attn(x, x, x,mask)
        # attn_output shape [Batch_Size X Seq_len X Embedding Dim]
        x = self.norm1(x + self.dropout(attn_output))
        # x shape [Batch_Size X Seq_len X Embedding Dim]
        ff_output = self.feed_forward(x)
        # ff_output shape [Batch_Size X Seq_len X Embedding Dim]
        x = self.norm2(x + self.dropout(ff_output))
        # x shape [Batch_Size X Seq_len X Embedding Dim]
        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x,tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x,enc_output,enc_output,src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [27]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length,device=device), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [47]:
src_vocab_size = len(source_vocab)
tgt_vocab_size = len(target_vocab)
input_dim = 256
num_heads = 4
num_layers = 6
d_ff = 2048
max_seq_length = get_max_seqlen()
dropout = 0.05
num_workers = 2
num_epochs = 15

model = Transformer(src_vocab_size, tgt_vocab_size, input_dim, num_heads, num_layers, d_ff, max_seq_length, dropout)
print(model)

Max length in dataset  46
Transformer(
  (encoder_embedding): Embedding(65916, 256)
  (decoder_embedding): Embedding(65916, 256)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=256, out_features=256, bias=True)
        (W_k): Linear(in_features=256, out_features=256, bias=True)
        (W_v): Linear(in_features=256, out_features=256, bias=True)
        (W_o): Linear(in_features=256, out_features=256, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=256, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.05, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLa

In [48]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

68052348


In [49]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

In [50]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

In [ ]:
source_dummy,target_dummy = next(iter(train_loader))

In [ ]:
print(source_dummy.shape,target_dummy.shape)

torch.Size([4, 34]) torch.Size([4, 8])


In [ ]:
print(source_dummy[1])

tensor([29829, 64754, 32864, 50366, 42869, 59298, 11704, 56936, 14787, 42869,
        59437, 56020,  3643, 17811, 51874, 58054, 44280, 14256, 53025,  1894,
         1775, 56020, 37102, 21582, 28600, 48104, 24424, 42943, 21132, 65257,
        60649, 14845, 42869, 59437])


In [ ]:
y_pred = model(source_dummy,target_dummy)
print(y_pred.shape,target_dummy.shape)

torch.Size([4, 8, 65916]) torch.Size([4, 8])


In [ ]:
y_pred = y_pred.reshape(-1,len(target_vocab))
target_dummy = target_dummy.reshape(-1)
print(y_pred.shape,target_dummy.shape)

torch.Size([32, 65916]) torch.Size([32])


In [41]:
# Check accuracy function
def check_accuracy(output,labels):
    _ , predpos = output.max(1)
    num_samples=len(labels)
    num_correct=(predpos==labels).sum()
    return (num_correct/num_samples)*100

# Save checkpoint
def save_checkpoint(state,filename='weights.pth.tar'):
    print('Saving weights-->')
    torch.save(state,filename)

# Load checkpoint
def load_checkpoint(checkpoint,model,optim):
    print('Loading weights-->')
    model.load_state_dict(checkpoint['state_dict'])
    optim.load_state_dict(checkpoint['optimizer'])

In [51]:
def train_loop(model,dataloader,loss_fun,optimizer,device):
    model.train()
    model.to(device)
    min_loss = None
    for epoch in range(num_epochs):
        losses = []
        accuracies = []
        loop = tqdm(enumerate(dataloader), total=len(dataloader), leave=True)
        for batch,(x,y) in loop:
            # put on cuda
            x = torch.tensor(x).to(device).long()
            y = torch.tensor(y).to(device).long()
            # x = x.to(device)
            # y = y.to(device)

            # forward pass
            y_pred = model(x,y)

            # calculate loss & accuracy
            loss = loss_fun(y_pred.reshape(-1,len(target_vocab)),y.reshape(-1))
            losses.append(loss.detach().item())

            accuracy = check_accuracy(y_pred.reshape(-1,len(target_vocab)),y.reshape(-1))
            accuracies.append(accuracy.detach().item())

            # zero out prior gradients
            optimizer.zero_grad()

            # backprop
            loss.backward()

            # update weights
            optimizer.step()
            scheduler.step()

            # Update TQDM progress bar
            loop.set_description(f"Epoch [{epoch}/{num_epochs}] ")
            loop.set_postfix(loss=loss.detach().item(), accuracy=accuracy.detach().item())

        moving_loss = sum(losses) / len(losses)
        moving_accuracy = sum(accuracies) / len(accuracies)
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        # Save check point
        if min_loss == None:
            min_loss = moving_loss
            save_checkpoint(checkpoint)
        elif moving_loss < min_loss:
            min_loss = moving_loss
            save_checkpoint(checkpoint)
        print('Epoch {0} : Loss = {1} , Training Accuracy={2}'.format(epoch, moving_loss, moving_accuracy))

In [52]:
train_loop(model,train_loader,criterion,optimizer,device)

Epoch [0/15] : 100%|██████████| 1536/1536 [02:33<00:00,  9.99it/s, accuracy=65.3, loss=0.742]


Saving weights-->
Epoch 0 : Loss = 2.283001027535647 , Training Accuracy=53.45129730185727


Epoch [1/15] : 100%|██████████| 1536/1536 [02:36<00:00,  9.83it/s, accuracy=73.4, loss=1.15]


Saving weights-->
Epoch 1 : Loss = 0.5832049241871573 , Training Accuracy=63.929689303040504


Epoch [2/15] : 100%|██████████| 1536/1536 [02:36<00:00,  9.81it/s, accuracy=65.7, loss=0.31]


...

Epoch 12 : Loss = 0.5899323651489491 , Training Accuracy=64.04612274716298


Epoch [13/15] : 100%|██████████| 1536/1536 [02:37<00:00,  9.78it/s, accuracy=77.3, loss=0.477]

Epoch 13 : Loss = 0.5903360002848785 , Training Accuracy=64.10921336710453



Epoch [14/15] : 100%|██████████| 1536/1536 [02:36<00:00,  9.79it/s, accuracy=60.1, loss=0.631]

Epoch 14 : Loss = 0.5937628798710648 , Training Accuracy=63.893871046602726


In [58]:
## Inference
def test_loop(model,dataloader,loss_fun,device):
    model.eval()
    model.to(device)
    losses = []
    samples,correct = 0,0
    loop = tqdm(enumerate(dataloader), total=len(dataloader), leave=True)
    with torch.no_grad():
        for batch,(x,y) in loop:
            # put on cuda
            x = torch.tensor(x).to(device).long()
            y = torch.tensor(y).to(device).long()

            # forward pass
            y_pred = model(x,y)

            # caclulate test loss
            loss = loss_fun(y_pred.reshape(-1,len(target_vocab)),y.reshape(-1))
            losses.append(loss.detach().item())

            # accuracy over entire dataset
            _,predpos=y_pred.reshape(-1,len(target_vocab)).max(1)
            samples+=len(y.reshape(-1))
            correct+=(predpos==y.reshape(-1)).sum().item()

            # Update TQDM progress bar
            loop.set_postfix(loss=loss.item())

    print("Final Test Accuracy = ",100 * (correct/samples))


In [59]:
test_loop(model,test_loader,criterion,device)

100%|██████████| 71/71 [00:02<00:00, 32.91it/s, loss=0.696]

Final Test Accuracy =  65.29104022754979
